<a href="https://colab.research.google.com/github/SpiceTrack/machine-learning-spicetrack/blob/mtauf-ml-arch/rempah_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Deep Learning Library
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

# Import Pretrained Model Library
from tensorflow.keras.callbacks import ModelCheckpoint

# Import Dataset Preprocessing Library
import scipy.io
import numpy as np
import pandas as pd

from google.colab import drive

import matplotlib.pyplot as plt
import numpy as np
import os

print(tf.__version__)

2.17.1


In [2]:
drive.mount('/content/drive', force_remount=True)

TRAIN_DIR = "/content/drive/MyDrive/Capstone/Project/Dataset/train"
VAL_DIR = "/content/drive/MyDrive/Capstone/Project/Dataset/val"
VAL_DIR = "/content/drive/MyDrive/Capstone/Project/Dataset/test"

BATCH_SIZE = 16
IMG_SIZE = (150, 150)
NUM_CLASSES = 31

Mounted at /content/drive


In [3]:
train_dataset = (tf.keras.utils.image_dataset_from_directory(TRAIN_DIR,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            label_mode='categorical',
                                                            seed=90))

Found 4557 files belonging to 31 classes.


In [4]:
val_dataset = tf.keras.utils.image_dataset_from_directory(VAL_DIR,
                                                          shuffle=True,
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE,
                                                          label_mode='categorical',
                                                          seed=90)

Found 992 files belonging to 31 classes.


In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(VAL_DIR,
                                                          shuffle=True,
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE,
                                                          label_mode='categorical',
                                                          seed=90)

Found 992 files belonging to 31 classes.


In [5]:
print(train_dataset.class_names)

['adas', 'andaliman', 'asam jawa', 'bawang bombai', 'bawang merah', 'bawang putih', 'biji ketumbar', 'bukan rempah', 'bunga lawang', 'cengkeh', 'daun jeruk', 'daun kemangi', 'daun ketumbar', 'daun salam', 'jahe', 'jinten', 'kapulaga', 'kayu manis', 'kayu secang', 'kemiri', 'kemukus', 'kencur', 'kluwek', 'kunyit', 'lada', 'lengkuas', 'pala', 'saffron', 'serai', 'vanili', 'wijen']


In [ ]:
def create_augmentation_model():

    augmentation_model = tf.keras.Sequential([
        tf.keras.Input(shape=(150, 150, 3)),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
        tf.keras.layers.RandomZoom(0.2)
    ])

    return augmentation_model

In [ ]:
def create_pre_trained_model():

    pre_trained_model = tf.keras.applications.InceptionResNetV2(
        include_top=False,
        input_shape=(150,150,3),
        weights='imagenet'
    )

    # Make all the layers in the pre-trained model non-trainable
    pre_trained_model.trainable = False

    return pre_trained_model

In [ ]:
def create_model():

    augmentation_layers = create_augmentation_model()
    pre_trained_model = create_pre_trained_model()

    inputs = tf.keras.Input(shape=(150,150,3))
    augmented = augmentation_layers(inputs)
    rescaled = tf.keras.layers.Rescaling(1./255)(augmented)

    x = pre_trained_model(rescaled)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    predictions = tf.keras.layers.Dense(31, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005),
        loss="categorical_crossentropy",
        metrics=['accuracy']
    )

    return model

In [ ]:
# Save your model in a variable
# tf.keras.backend.set_learning_phase(1)
model = create_model()
print(model.summary())

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_resnet_v2 (Functional)     │ (None, 3, 3, 1536)          │      54,336,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 13824)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │      14,156,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 31)                  │           2,015 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 69,192,831 (263.95 MB)

 Trainable params: 14,856,095 (56.67 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

None
There are 69,192,831 total parameters in this model.
There are 14,856,095 trainable parameters in this model.


In [ ]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_accuracy']>0.1:
            self.model.stop_training = True
            print("\nReached 50% accuracy so cancelling training!")

In [ ]:
history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = 100,
    verbose = 1,
    callbacks = [EarlyStoppingCallback()],
)

Epoch 1/100
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0923 - loss: 4.2263

In [ ]:
import matplotlib.pyplot as plt

# Plot the training and validation accuracies for each epoch

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.show()

In [ ]:
# tf.saved_model.save(history, "savedmodels/1")
# model.save("/content/drive/MyDrive/Capstone/Project/savedmodels/3keras/model.keras",)
tf.saved_model.save(model, "/content/drive/MyDrive/Capstone/Project/savedmodels/inceptionresnet")

In [ ]:
 !pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='MobilenetV1/Predictions/Reshape_1' \
    --saved_model_tags=serve \
    /content/drive/MyDrive/Capstone/Project/savedmodels/2 \
    /content/drive/MyDrive/Capstone/Project/savedmodels/web_model